# **VIRTUAL RACE ENGINEER**

In [175]:
import fastf1 as ff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1.plotting
fastf1.plotting.setup_mpl()
import fastf1.api as fap
fastf1.Cache.enable_cache(r"D:\Prabhu\SEM 7\F1 Data")
import re
import datetime as dt
import fastf1.mvapi as famp


### LOADING SESSION

In [176]:
session = ff.get_session(2023,'Dutch','R')
session.load()

core           INFO 	Loading data for Dutch Grand Prix - Race [v3.3.7]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '14', '10', '11', '55', '44', '4', '23', '81', '31', '18', '27', '40', '77', '22', '20', '63', '24', '16', '2']


In [177]:
laps=session.laps
drivers=laps['Driver'].unique()

### FETCHING TELEMETRY DATA 

In [178]:
total_telemetry=[]
for i in drivers:
    drivers_laps=laps.pick_driver(i)
    driver_telemetry=drivers_laps.get_car_data()
    
    driver_telemetry['Driver']=i
    total_telemetry.append(driver_telemetry)
telemetry_data=pd.concat(total_telemetry,ignore_index=True)

### FETCHING WEATHER,TYRE,LAP AND POSITION DATA 

In [179]:
weather_data=pd.DataFrame(fap.weather_data(session.api_path))
tyre_data = laps[['Driver', 'LapNumber', 'Compound', 'FreshTyre','TyreLife']]
lap_data=laps[['Time','Driver','DriverNumber','LapNumber','LapTime','Sector1Time','Sector2Time','Sector3Time','IsAccurate']]
position_data=laps[['Driver','DriverNumber','LapNumber','Position','LapStartTime']]

req            INFO 	Using cached data for weather_data


### FETCHING RACE CONTROL MESSAGES

In [180]:
rcm_data = pd.DataFrame(fap.race_control_messages(session.api_path))
msg=rcm_data[['Time','Status','Message']]

#Filter for Yellow, Red, VSC, SC 
yellow_flags_key=['YELLOW','DOUBLE YELLOW']
red_flags_key=['RED']
vsc_key=['VSC','VIRTUAL SAFETY CAR']
sc_key=['SC','SAFETY CAR']

#Searching using the keyword in REGEX
yellow_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in yellow_flags_key))]
red_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in red_flags_key))]
vsc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in vsc_key))]
sc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in sc_key))]


req            INFO 	Using cached data for race_control_messages


### FETCHING LAP DATA (PITS STOPS, DRIVER POSITION ETC.)

In [181]:
lapsdata, streamdata = fap.timing_data(session.api_path)
# pd.set_option('display.max_columns', None)
lapsdata=pd.DataFrame(lapsdata)
lapsdata['DriverNumber'] = lapsdata['Driver']
gapdata=pd.DataFrame(streamdata)
gapdata['DriverNumber'] = gapdata['Driver']
gap_data = gapdata[['Time','DriverNumber','Position','GapToLeader','IntervalToPositionAhead']]
laps_data=lapsdata[['Time','DriverNumber','LapTime','NumberOfPitStops','PitOutTime','PitInTime']]

req            INFO 	Using cached data for _extended_timing_data


### GETTING CORNERS INFO

In [182]:
circuit_info=session.get_circuit_info()
corners_data=circuit_info.corners

### MAKING A MASTER DATA (POSITION DATA, POSITION DATA, LAPS DATA)
###### NOTE:WE ARE KEEPING THE GAP DATA, WEATHER DATA, RACE CONTROL MESSAGES AND THE TELEMETRY DATA SEPERATE BECAUSE EACH DATA IS GENERATED IN A COMPLETELY DIFFERENT TIME

In [183]:
master_data = lap_data.merge(position_data, on=['Driver', 'DriverNumber', 'LapNumber'], how='left')
master_data = master_data.merge(tyre_data, on=['Driver', 'LapNumber'], how='left')
master_data = master_data.merge(laps_data, on=['DriverNumber','Time','LapTime'], how='left')

### CONVERTING TIME TO SECONDS

In [184]:
master_data['Sector1Time(s)']=master_data['Sector1Time'].dt.total_seconds()
master_data['Sector2Time(s)']=master_data['Sector2Time'].dt.total_seconds()
master_data['Sector3Time(s)']=master_data['Sector3Time'].dt.total_seconds()
master_data['Cum.SectorTime(s)']=master_data['Sector1Time(s)']+master_data['Sector2Time(s)']+master_data['Sector3Time(s)']
master_data['LapTime(s)']=master_data['LapTime'].dt.total_seconds()
master_data['LapStartTime(s)']=master_data['LapStartTime'].dt.total_seconds()
master_data['PitInTime(s)']=master_data['PitInTime'].dt.total_seconds()
master_data['PitOutTime(s)']=master_data['PitOutTime'].dt.total_seconds().shift(-1)

#### CALCULATING PITSTOP TIME AND LAP DELTA

In [188]:
master_data['PitStopTime(s)']=master_data['PitOutTime(s)']-master_data['PitInTime(s)']
master_data['PrevLapTime(s)']=master_data['LapTime(s)'].shift(1)
master_data['LapΔ(s)']=master_data['LapTime(s)']-master_data['PrevLapTime(s)']
master_data

,Time,Driver,DriverNumber,LapNumber,LapTime,Sector1Time,Sector2Time,Sector3Time,IsAccurate,Position,LapStartTime,Compound,FreshTyre,TyreLife,NumberOfPitStops,PitOutTime,PitInTime,Sector1Time(s),Sector2Time(s),Sector3Time(s),Cum.SectorTime(s),LapTime(s),LapStartTime(s),PitInTime(s),PitOutTime(s),PitStopTime(s),PrevLapTime(s),LapΔ(s)
0,0 days 01:03:36.820000,VER,1,1.0,0 days 00:01:31.585000,NaT,0 days 00:00:28.929000,0 days 00:00:30.478000,False,1.0,0 days 01:02:04.960000,SOFT,True,1.0,NaN,NaT,NaT,NaN,28.929,30.478,NaN,91.585,3724.960,NaN,NaN,NaN,NaN,NaN
1,0 days 01:05:26.792000,VER,1,2.0,0 days 00:01:49.972000,0 days 00:00:36.330000,0 days 00:00:36.685000,0 days 00:00:36.957000,False,3.0,0 days 01:03:36.820000,SOFT,True,2.0,0.0,NaT,0 days 01:05:25.519000,36.330,36.685,36.957,109.972,109.972,3816.820,3925.519,3945.581,20.062,91.585,18.387
2,0 days 01:07:15.450000,VER,1,3.0,0 days 00:01:48.658000,0 days 00:00:49.822000,0 days 00:00:30.905000,0 days 00:00:27.931000,False,5.0,0 days 01:05:26.792000,INTERMEDIATE,True,1.0,1.0,0 days 01:05:45.581000,NaT,49.822,30.905,27.931,108.658,108.658,3926.792,NaN,NaN,NaN,109.972,-1.314
3,0 days 01:08:40.701000,VER,1,4.0,0 days 00:01:25.251000,0 days 00:00:29.996000,0 days 00:00:29.479000,0 days 00:00:25.776000,True,4.0,0 days 01:07:15.450000,INTERMEDIATE,True,2.0,1.0,NaT,NaT,29.996,29.479,25.776,85.251,85.251,4035.450,NaN,NaN,NaN,108.658,-23.407
4,0 days 01:10:04.162000,VER,1,5.0,0 days 00:01:23.461000,0 days 00:00:28.478000,0 days 00:00:29.064000,0 days 00:00:25.919000,True,4.0,0 days 01:08:40.701000,INTERMEDIATE,True,3.0,1.0,NaT,NaT,28.478,29.064,25.919,83.461,83.461,4120.701,NaN,NaN,NaN,85.251,-1.790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,0 days 03:20:55.371000,PIA,81,68.0,0 days 00:01:25.004000,0 days 00:00:29.890000,0 days 00:00:29.060000,0 days 00:00:26.054000,True,9.0,0 days 03:19:30.367000,INTERMEDIATE,False,9.0,5.0,NaT,NaT,29.890,29.060,26.054,85.004,85.004,11970.367,NaN,NaN,NaN,88.541,-3.537
1339,0 days 03:22:18.282000,PIA,81,69.0,0 days 00:01:22.911000,0 days 00:00:28.834000,0 days 00:00:28.632000,0 days 00:00:25.445000,True,9.0,0 days 03:20:55.371000,INTERMEDIATE,False,10.0,5.0,NaT,NaT,28.834,28.632,25.445,82.911,82.911,12055.371,NaN,NaN,NaN,85.004,-2.093
1340,0 days 03:23:40.941000,PIA,81,70.0,0 days 00:01:22.659000,0 days 00:00:28.662000,0 days 00:00:28.457000,0 days 00:00:25.540000,True,9.0,0 days 03:22:18.282000,INTERMEDIATE,False,11.0,5.0,NaT,NaT,28.662,28.457,25.540,82.659,82.659,12138.282,NaN,NaN,NaN,82.911,-0.252
1341,0 days 03:25:03.237000,PIA,81,71.0,0 days 00:01:22.296000,0 days 00:00:28.423000,0 days 00:00:28.419000,0 days 00:00:25.454000,True,9.0,0 days 03:23:40.941000,INTERMEDIATE,False,12.0,5.0,NaT,NaT,28.423,28.419,25.454,82.296,82.296,12220.941,NaN,NaN,NaN,82.659,-0.363
